<a href="https://colab.research.google.com/github/pratyaksh29/Discord-Bot/blob/master/AI_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow


In [ ]:
!pip install tflearn
!pip install nltk

In [ ]:
!pip install discord

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving intents.json to intents (2).json


In [ ]:
bruh = next(iter(uploaded.values()))
type(bruh)

bytes

In [ ]:
import nltk
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
import nest_asyncio
nest_asyncio.apply()
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()
import numpy
import tflearn
import tensorflow
import random
import json
import discord
from discord.ext import commands
import discord
from discord.ext import commands
from discord.utils import get
import random
from discord.ext.commands import has_permissions
import pickle
prefix = '.'
client = commands.Bot(command_prefix = prefix)


with open('intents.json') as file:
    data = json.loads(bruh.decode())


words = []
labels = []
docs_x = []
docs_y = []

for intent in data['intents']:
    for pattern in intent['patterns']:
        wrds = nltk.word_tokenize(pattern)
        words.extend(wrds)
        docs_x.append(wrds)
        docs_y.append(intent["tag"])
        
    if intent['tag'] not in labels:
        labels.append(intent['tag'])

words = [stemmer.stem(w.lower()) for w in words if w != "?"]
words = sorted(list(set(words)))

labels = sorted(labels)

#twitter sentiment analysis 
#clean data



training = []
output = []

out_empty = [0 for i in range(len(labels))]

for x, doc in enumerate(docs_x):
    bag = []

    wrds = [stemmer.stem(w.lower()) for w in doc]

    for w in words:
        if w in wrds:
            bag.append(1)
        else:
            bag.append(0)
    output_row = out_empty[:]
    output_row[labels.index(docs_y[x])] = 1 #we will look through labels list and see where the tag is and set that value to 1

    training.append(bag)
    output.append(output_row)


training = numpy.array(training)
output = numpy.array(output)

import tensorflow as tf  
try:
    model.load('model.tflearn')
except:
    tf.compat.v1.reset_default_graph()

    net = tflearn.input_data(shape=[None, len(training[0])])
    net = tflearn.fully_connected(net, 8)
    net = tflearn.fully_connected(net, 8)
    net = tflearn.fully_connected(net, len(output[0]), activation='softmax')
    net = tflearn.regression(net)

    model = tflearn.DNN(net)

    model.fit(training, output, n_epoch=1000, batch_size=8, show_metric=True)
    model.save("model.tflearn")


def bag_of_words(s, words):
    bag = [0 for j in range(len(words))]

    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]

    for idk in s_words:
        for i, w in enumerate(words):
            if w == idk:
                bag[i] = 1
            
    return numpy.array(bag)

@client.event
  
async def on_message(msg):
  if msg.author == client.user:
    return 
  results = model.predict([bag_of_words(msg.content, words)])[0]
  results_index = numpy.argmax(results) #index of greatest value in our list
  tag = labels[results_index]
  
  if results[results_index] > 0.8:
    for tg in data["intents"]:
      if tg['tag'] == tag:
          responses = tg['responses']
    await msg.channel.send(random.choice(responses))   
  else:
    await msg.channel.send("I'm not sure, try again or ask a different questionh")

@client.event
async def on_ready():
    await client.change_presence(status=discord.Status.online, activity=discord.Game('Ai Chatbot'))
print("bot online")    
client.run('TOKEN')


Training Step: 4999  | total loss: 1.70257 | time: 0.019s
| Adam | epoch: 1000 | loss: 1.70257 - acc: 0.8926 -- iter: 32/39
Training Step: 5000  | total loss: 1.53530 | time: 0.025s
| Adam | epoch: 1000 | loss: 1.53530 - acc: 0.9033 -- iter: 39/39
--
INFO:tensorflow:/content/model.tflearn is not in all_model_checkpoint_paths. Manually adding it.
